In [1]:
from google.colab import files
uploaded = files.upload()

Saving fra.txt to fra.txt


In [2]:
import tensorflow
from keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense
import numpy as np
batch_size=64
epochs=100
latent_dim=512
data_path='fra.txt'

In [3]:
input_texts=[]
target_texts=[]
input_char=set()
target_char=set()
with open(data_path,'r',encoding='utf-8')as f:
  lines=f.read().split('\n')
for line in lines[:10000]:
  input_text,target_text,_=line.split('\t')
  target_text='\t'+target_text+'\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_char:
      input_char.add(char)
  for char in target_text:
    if char not in target_char:
      target_char.add(char)



In [4]:
input_char

{' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [5]:
target_char

{'\t',
 '\n',
 ' ',
 '!',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '’',
 '\u202f'}

In [6]:
len(input_texts)

10000

In [7]:
input_char=sorted(input_char)
target_char=sorted(target_char)
num_encoder_token=len(input_char)
num_decoder_token=len(target_char)
max_encoder_seq_len=max([len(txt)for txt in input_texts])
max_decoder_seq_len=max([len(txt)for txt in target_texts])

In [8]:
num_encoder_token

70

In [9]:
num_decoder_token

93

In [10]:
max_encoder_seq_len

14

In [11]:
max_decoder_seq_len

59

In [12]:
 input_texts[229]

'Come on!'

In [13]:
target_texts[229]

'\tSecouez-vous !\n'

In [14]:
input_token=dict((char,i) for i,char in enumerate(input_char))

In [15]:
input_token

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69}

In [16]:
target_token=dict((char,i) for i,char in enumerate(target_char))

In [17]:
target_token

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 ',': 9,
 '-': 10,
 '.': 11,
 '0': 12,
 '1': 13,
 '2': 14,
 '3': 15,
 '5': 16,
 '8': 17,
 '9': 18,
 ':': 19,
 '?': 20,
 'A': 21,
 'B': 22,
 'C': 23,
 'D': 24,
 'E': 25,
 'F': 26,
 'G': 27,
 'H': 28,
 'I': 29,
 'J': 30,
 'K': 31,
 'L': 32,
 'M': 33,
 'N': 34,
 'O': 35,
 'P': 36,
 'Q': 37,
 'R': 38,
 'S': 39,
 'T': 40,
 'U': 41,
 'V': 42,
 'W': 43,
 'Y': 44,
 'a': 45,
 'b': 46,
 'c': 47,
 'd': 48,
 'e': 49,
 'f': 50,
 'g': 51,
 'h': 52,
 'i': 53,
 'j': 54,
 'k': 55,
 'l': 56,
 'm': 57,
 'n': 58,
 'o': 59,
 'p': 60,
 'q': 61,
 'r': 62,
 's': 63,
 't': 64,
 'u': 65,
 'v': 66,
 'w': 67,
 'x': 68,
 'y': 69,
 'z': 70,
 '\xa0': 71,
 '«': 72,
 '»': 73,
 'À': 74,
 'Ç': 75,
 'É': 76,
 'Ê': 77,
 'à': 78,
 'â': 79,
 'ç': 80,
 'è': 81,
 'é': 82,
 'ê': 83,
 'î': 84,
 'ï': 85,
 'ô': 86,
 'ù': 87,
 'û': 88,
 'œ': 89,
 '\u2009': 90,
 '’': 91,
 '\u202f': 92}

In [18]:
len(input_texts)

10000

In [19]:
encoder_input_data=np.zeros((len(input_texts),max_encoder_seq_len,num_encoder_token),dtype='float32')

In [20]:
encoder_input_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [21]:
decoder_input_data=np.zeros((len(input_texts),max_decoder_seq_len,num_decoder_token),dtype='float32')

In [22]:
decoder_target_data=np.zeros((len(input_texts),max_decoder_seq_len,num_decoder_token),dtype='float32')

In [23]:
for i ,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
  for t,char in enumerate(input_text):
    encoder_input_data[i,t,input_token[char]]=1
  encoder_input_data[i,t+1:,input_token[' ']]=1
  for t,char in enumerate(target_text):
    decoder_input_data[i,t,target_token[char]]=1
    if t>0:
      decoder_target_data[i,t-1,target_token[char]]=1
  decoder_input_data[i,t+1:,target_token[' ']]=1
  decoder_target_data[i,t:,target_token[' ']]=1


In [24]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_token))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_token))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_token, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [25]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 8s 29ms/step - loss: 1.2046 - accuracy: 0.7328 - val_loss: 1.0273 - val_accuracy: 0.7157
Epoch 2/100
125/125 [==============================] - 2s 20ms/step - loss: 0.8114 - accuracy: 0.7768 - val_loss: 0.8081 - val_accuracy: 0.7734
Epoch 3/100
125/125 [==============================] - 2s 20ms/step - loss: 0.7480 - accuracy: 0.8037 - val_loss: 0.7344 - val_accuracy: 0.7918
Epoch 4/100
125/125 [==============================] - 3s 24ms/step - loss: 0.6086 - accuracy: 0.8249 - val_loss: 0.6491 - val_accuracy: 0.8119
Epoch 5/100
125/125 [==============================] - 2s 20ms/step - loss: 0.5569 - accuracy: 0.8376 - val_loss: 0.6204 - val_accuracy: 0.8218
Epoch 6/100
125/125 [==============================] - 2s 20ms/step - loss: 0.5243 - accuracy: 0.8458 - val_loss: 0.5817 - val_accuracy: 0.8296
Epoch 7/100
125/125 [==============================] - 2s 20ms/step - loss: 0.4967 - accuracy: 0.8539 - val_loss: 0.5607 - val_accuracy:

In [26]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
reverse_input_char_index=dict((i,char) for char,i in input_token.items())
reverse_target_char_index=dict((i,char) for char,i in target_token.items())

In [27]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_token))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_len):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_token))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [28]:
input_text="Hello!"

In [29]:
input_seq = np.zeros((1, max_encoder_seq_len, num_encoder_token), dtype='float32')
for t, char in enumerate(input_text):
    input_seq[0, t, input_token[char]] = 1
input_seq[0, len("input_text"):, input_token[' ']] = 1

In [30]:
decoded_sentence = decode_sequence(input_seq)

1/1 [==============================] - 0s 18ms/step


In [31]:
decoded_sentence

'Salut !\n'

In [38]:
for index in range(1000,1010):
  input=encoder_input_data[index:index+1]
  decoded=decode_sequence(input)
  print('input',input_texts[index])
  print('decoded',decoded)

1/1 [==============================] - 0s 20ms/step
input Go.
decoded Va !

1/1 [==============================] - 0s 19ms/step
input Go.
decoded Va !

1/1 [==============================] - 0s 17ms/step
input Go.
decoded Va !

1/1 [==============================] - 0s 19ms/step
input Go.
decoded Va !

1/1 [==============================] - 0s 20ms/step
input Hi.
decoded Salut.

1/1 [==============================] - 0s 19ms/step
input Hi.
decoded Salut.

1/1 [==============================] - 0s 18ms/step
input Run!
decoded Fuyons !

1/1 [==============================] - 0s 21ms/step
input Run!
decoded Fuyons !

1/1 [==============================] - 0s 17ms/step
input Run!
decoded Fuyons !

1/1 [==============================] - 0s 18ms/step
input Run!
decoded Fuyons !

1/1 [==============================] - 0s 27ms/step
input Run!
decoded Fuyons !

1/1 [==============================] - 0s 33ms/step
input Run!
decoded Fuyons !

1/1 [==============================] - 0s 19ms/step
in